# Globus Connect Personal - Using `pexpect` to setup.

---

There was an issue with trying to setup the Globus Connect Personal (GCP) client/application/process from a Jupyter notebook that runs on a server that does not perform Globus autentication prior to starting the kernel. Below, is a simple script to allow a secondary input of the authentication code that is retrieved from Globus upon setup.

In [ ]:
# obtain the latest GCP (assuming Linux OS)
!wget --no-check-certificate https://downloads.globus.org/globus-connect-personal/linux/stable/globusconnectpersonal-latest.tgz

In [ ]:
!tar xzf globusconnectpersonal-latest.tgz

Now that we have obtained the latext application release, the following will allow you to setup the client.

In [ ]:
import sys
import time
import glob
import pexpect

In [ ]:
# prevent a rerun of the expect which would deadlock the process
def get_input(child, stop_string, result):
    if not result:
        child.expect(stop_string)
        result = input()
    else:
        print('Input already entered: {}'.format(result))
    
    return result

In [ ]:
auth_code = ''
endpoint_name = ''

# obtain folder name (remove archive and select first one found)
folder = [d for d in glob.glob('globusconnectpersonal-*') if '.t' not in d][0]

# start GCP setup and pipe child output to stdout
child = pexpect.spawn(folder+'/globusconnect -setup', encoing='utf-8', echo=False)
child.logfile_read = sys.stdout

In [ ]:
child.sendline(auth_code)

In [ ]:
# GCP needs a name for the endpoint
endpoint_name = get_input(child, 'Name: ', endpoint_name)

In [ ]:
child.sendline(endpoint_name)

In [ ]:
child.expect(pexpect.EOF)
child.close

In [ ]:
# get endpoint ID
print(child.before.split('id: ')[1].split('\r')[0])